# Propensity Score Matching

**Author:** Daniel Cavalli <br>
**Last Update:** 2024-11-08

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

In [ ]:
def prepare_data_for_psm(df):
    """
    Enhanced data preparation with additional features and better handling of outliers
    """
    # Create lists of column names by type
    precip_cols = [col for col in df.columns if 'precipitacao' in col]
    pressao_cols = [col for col in df.columns if 'pressao' in col]
    temp_cols = [col for col in df.columns if 'temperatura' in col]
    umidade_cols = [col for col in df.columns if 'umidade' in col]
    vento_cols = [col for col in df.columns if 'vento' in col]
    
    # Create copy of dataframe to prevent unintended changes
    df_clean = df.copy()
    
    # Calculate pre-treatment averages for production
    df_clean['avg_pre_production'] = df_clean.groupby('id_microrregiao')['total_valor_producao'].transform(
        lambda x: x.expanding().mean()
    )
    
    # Create imputer with median strategy (more robust to outliers)
    imputer = SimpleImputer(strategy='median')
    
    # Impute missing values for each group of climate variables
    for cols in [precip_cols, pressao_cols, temp_cols, umidade_cols, vento_cols]:
        if cols:  # Check if list is not empty
            df_clean[cols] = imputer.fit_transform(df_clean[cols])
    
    # Calculate yearly averages with outlier capping
    for prefix, cols in [
        ('avg_precip', precip_cols),
        ('avg_pressao', pressao_cols),
        ('avg_temp', temp_cols),
        ('avg_umidade', umidade_cols),
        ('avg_vento', vento_cols)
    ]:
        if cols:
            # Calculate mean
            df_clean[prefix] = df_clean[cols].mean(axis=1)
            
            # Cap outliers at 1st and 99th percentiles
            lower = np.percentile(df_clean[prefix], 1)
            upper = np.percentile(df_clean[prefix], 99)
            df_clean[prefix] = df_clean[prefix].clip(lower, upper)
    
    # Add year fixed effects
    df_clean['year_effect'] = df_clean.groupby('ano')['total_valor_producao'].transform('mean')
    
    # Add region-specific trends
    df_clean['region_trend'] = df_clean.groupby('id_microrregiao')['total_valor_producao'].transform(
        lambda x: np.arange(len(x))
    )
    
    df_clean.dropna(inplace=True)
    return df_clean

In [ ]:
def get_treatment_events(df):
    """
    Identify all treatment events (when a region first receives treatment)
    """
    treatment_events = []
    
    # Get all treated regions
    treated_regions = df[df['treatment'] == 1]['id_microrregiao'].unique()
    
    for region in treated_regions:
        region_data = df[df['id_microrregiao'] == region].sort_values('ano')
        # Find first year of treatment
        treatment_year = region_data[region_data['treatment'] == 1]['ano'].min()
        
        treatment_events.append({
            'region': region,
            'treatment_year': treatment_year
        })
    
    return pd.DataFrame(treatment_events)

In [ ]:
def match_for_treatment_event(df, treated_region, treatment_year, features, caliper=0.1):  # Reduced caliper
    """
    Enhanced matching with stricter criteria and exact matching on year
    """
    # Get data up to treatment year
    df_period = df[df['ano'] <= treatment_year].copy()
    
    # Create treatment indicator for this specific matching
    df_period['current_treatment'] = ((df_period['id_microrregiao'] == treated_region) & 
                                    (df_period['ano'] == treatment_year)).astype(int)
    
    # Get potential controls from the same year or one year before/after
    potential_controls = df_period[
        (df_period['id_microrregiao'] != treated_region) & 
        (df_period['ano'].between(treatment_year - 1, treatment_year + 1)) &
        (df_period['treatment'] == 0)
    ]
    
    # Get treated observation
    treated_obs = df_period[df_period['current_treatment'] == 1]
    
    if len(treated_obs) == 0 or len(potential_controls) == 0:
        return None
    
    # Prepare data for propensity score calculation
    matching_data = pd.concat([treated_obs, potential_controls])
    
    # Standardize features
    scaler = StandardScaler()
    X = scaler.fit_transform(matching_data[features])
    
    # Calculate propensity scores with balanced class weights
    model = LogisticRegression(random_state=42, class_weight='balanced')
    model.fit(X, matching_data['current_treatment'])
    propensity_scores = model.predict_proba(X)[:, 1]
    
    # Add propensity scores to the data
    matching_data['propensity_score'] = propensity_scores
    
    # Get treated unit's propensity score
    treated_score = matching_data[matching_data['current_treatment'] == 1]['propensity_score'].iloc[0]
    
    # Calculate distances for all potential controls
    potential_controls = matching_data[matching_data['current_treatment'] == 0].copy()
    potential_controls['distance'] = abs(potential_controls['propensity_score'] - treated_score)
    
    # Apply stricter caliper
    caliper_threshold = caliper * np.std(matching_data['propensity_score'])
    potential_controls = potential_controls[potential_controls['distance'] <= caliper_threshold]
    
    # Select fewer, better matches
    matches = potential_controls.nsmallest(n=3, columns='distance')  # Reduced from 5 to 3
    
    if len(matches) == 0:
        return None
        
    # Add matching information
    matches['treated_region'] = treated_region
    matches['treatment_year'] = treatment_year
    matches['matched_control'] = 1
    
    treated_obs['treated_region'] = treated_region
    treated_obs['treatment_year'] = treatment_year
    treated_obs['matched_control'] = 0
    
    # Combine treated and matched controls
    matched_data = pd.concat([treated_obs, matches])
    
    return matched_data

In [ ]:
def perform_psm_matching(df):
    """
    Enhanced PSM matching with additional features
    """
    # Preparing data
    df_clean = prepare_data_for_psm(df)
    
    # Features for matching
    features = [
        'avg_precip', 
        'avg_pressao', 
        'avg_temp', 
        'avg_umidade', 
        'avg_vento',
        'avg_pre_production',
        'year_effect',
        'region_trend'
    ]
    
    # Treatment events
    treatment_events = get_treatment_events(df_clean)
    
    # Performing matching
    all_matches = []
    for _, event in treatment_events.iterrows():
        matches = match_for_treatment_event(
            df_clean,
            event['region'],
            event['treatment_year'],
            features
        )
        if matches is not None:
            all_matches.append(matches)
    
    if all_matches:
        matched_df = pd.concat(all_matches, ignore_index=True)
        matched_df['match_id'] = (matched_df['treated_region'].astype(str) + '_' + 
                                 matched_df['treatment_year'].astype(str))
        return matched_df
    else:
        return None

In [76]:
# Read data
df = pd.read_csv('../data/csv/PAM_MET_SOJA_pivoted.csv')

# Perform matching
matched_df = perform_psm_matching(df)

if matched_df is not None:
    print("\nMatching Summary:")
    print(f"Total treatment events: {matched_df['treated_region'].nunique()}")
    print(f"Total matched pairs: {len(matched_df[matched_df['matched_control'] == 1])}")
    
    # Save matched dataset for DiD analysis
    matched_df.to_csv('../data/matched_data_for_did.csv', index=False)
    
    # Print matching structure
    print("\nMatching Structure:")
    matching_summary = matched_df.groupby('match_id').agg({
        'matched_control': 'sum',
        'id_microrregiao': 'count'
    }).reset_index()
    print(matching_summary.head())
else:
    print("No valid matches found.")

C:\Users\Dan\AppData\Local\Temp\ipykernel_13464\3367785554.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treated_obs['treated_region'] = treated_region
C:\Users\Dan\AppData\Local\Temp\ipykernel_13464\3367785554.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treated_obs['treatment_year'] = treatment_year
C:\Users\Dan\AppData\Local\Temp\ipykernel_13464\3367785554.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in


Matching Summary:
Total treatment events: 42
Total matched pairs: 58

Matching Structure:
     match_id  matched_control  id_microrregiao
0  11001_2008                1                2
1  11003_2009                1                2
2  15015_2009                1                2
3  17004_2009                1                2
4  17005_2007                1                2


In [74]:
treated_production = matched_df[matched_df['treatment'] == 1]['total_valor_producao'].mean()
control_production = matched_df[matched_df['treatment'] == 0]['total_valor_producao'].mean()

print(f"Number of matched pairs: {len(matched_df) // 2}")
print(f"Average production value (treated): {treated_production:.2f}")
print(f"Average production value (control): {control_production:.2f}")

Number of matched pairs: 50
Average production value (treated): 2.62
Average production value (control): 2.65


In [77]:
un_treated_production = df[df['treatment'] == 1]['total_valor_producao'].mean()
un_control_production = df[df['treatment'] == 0]['total_valor_producao'].mean()

print(f"Number of matched pairs: {len(df) // 2}")
print(f"Average production value (treated): {un_treated_production:.2f}")
print(f"Average production value (control): {un_control_production:.2f}")

Number of matched pairs: 2805
Average production value (treated): 2.60
Average production value (control): 2.88
